In [1]:
import chess
import heapq

def score_position(board):
    if board.is_checkmate():
        return -10000 if board.turn else 10000

    values = {
        chess.PAWN: 100,
        chess.KNIGHT: 320,
        chess.BISHOP: 330,
        chess.ROOK: 500,
        chess.QUEEN: 900,
        chess.KING: 20000
    }

    score = 0
    for sq in chess.SQUARES:
        piece = board.piece_at(sq)
        if piece:
            val = values[piece.piece_type]
            score += val if piece.color == chess.WHITE else -val

    return score

def beam_search_moves(board, beam_width=3, max_depth=3):
    beam = [(score_position(board), [], board.copy())]

    for _ in range(max_depth):
        next_moves = []

        for val, sequence, curr_board in beam:
            if curr_board.is_game_over():
                next_moves.append((val, sequence, curr_board))
                continue

            for move in curr_board.legal_moves:
                temp_board = curr_board.copy()
                temp_board.push(move)
                eval_score = -score_position(temp_board)
                next_moves.append((eval_score, sequence + [move], temp_board))

        if not next_moves:
            break

        beam = heapq.nlargest(beam_width, next_moves, key=lambda x: x[0])

    if beam:
        final_score, best_seq, _ = beam[0]
        return best_seq, final_score

    return [], 0

def format_moves(moves):
    return " → ".join(move.uci() for move in moves)

def main():
    board = chess.Board()
    width = 3
    depth = 3

    best_moves, score = beam_search_moves(board, width, depth)

    print(board)

    if best_moves:
        print(f"Best sequence: {format_moves(best_moves)}")
        print(f"Score: {score}")
        print(f"Recommended move: {best_moves[0].uci()}")
    else:
        print("No moves found.")

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'chess'

In [ ]:
import random
import math
import time

def distance(p1, p2):
    return math.hypot(p1[0] - p2[0], p1[1] - p2[1])

def total_distance(route, coords):
    return sum(distance(coords[route[i]], coords[route[(i + 1) % len(route)]]) for i in range(len(route)))

def generate_neighbors(route):
    n = len(route)
    return [route[:i] + [route[j]] + route[i+1:j] + [route[i]] + route[j+1:] for i in range(n) for j in range(i+1, n)]

def hill_climb(coords, max_iters=1000):
    n = len(coords)
    current = list(range(n))
    random.shuffle(current)
    best_dist = total_distance(current, coords)

    for _ in range(max_iters):
        neighbors = generate_neighbors(current)
        next_route = min(neighbors, key=lambda r: total_distance(r, coords))
        next_dist = total_distance(next_route, coords)

        if next_dist < best_dist:
            current, best_dist = next_route, next_dist
        else:
            break

    return current, best_dist

def main():
    coords = [
        (25, 70), (70, 80), (100, 50), (30, 40),
        (10, 20), (75, 30), (60, 90), (40, 60),
        (90, 20), (50, 50), (20, 80), (95, 70)
    ]

    start = time.time()
    route, dist = hill_climb(coords)
    end = time.time()

    print("Optimized Route:", route)
    print(f"Total Distance: {dist:.2f}")
    print(f"Time Taken: {end - start:.2f}s")

if __name__ == "__main__":
    main()


Optimized Route: [1, 11, 2, 8, 5, 4, 3, 9, 7, 0, 10, 6]
Total Distance: 312.32
Time Taken: 0.00s


In [ ]:
import random
import math

NUM_CITIES = 10
POPULATION_SIZE = 50
ELITE_SIZE = 10
MUTATION_RATE = 0.01
MAX_GENERATIONS = 500

def generate_cities(num_cities=NUM_CITIES, seed=42):
    random.seed(seed)
    return [(random.randint(0, 100), random.randint(0, 100)) for _ in range(num_cities)]

def calculate_distance(c1, c2):
    return math.hypot(c1[0] - c2[0], c1[1] - c2[1])

def route_distance(route, cities):
    return sum(calculate_distance(cities[route[i]], cities[route[(i + 1) % len(route)]]) for i in range(len(route)))

def fitness(route, cities):
    dist = route_distance(route, cities)
    return 1 / dist if dist > 0 else float('inf')

def initialize_population(size=POPULATION_SIZE):
    base = list(range(NUM_CITIES))
    return [random.sample(base, len(base)) for _ in range(size)]

def rank_population(population, cities):
    return sorted(population, key=lambda r: fitness(r, cities), reverse=True)

def select(pop_ranked):
    elite = pop_ranked[:ELITE_SIZE]
    rest = random.choices(pop_ranked, weights=[fitness(r, cities) for r in pop_ranked], k=POPULATION_SIZE - ELITE_SIZE)
    return elite + rest

def crossover(p1, p2):
    start, end = sorted(random.sample(range(len(p1)), 2))
    child = [-1] * len(p1)
    child[start:end+1] = p1[start:end+1]
    fill = [c for c in p2 if c not in child]
    pointer = 0
    for i in range(len(child)):
        if child[i] == -1:
            child[i] = fill[pointer]
            pointer += 1
    return child

def mutate(route):
    for i in range(len(route)):
        if random.random() < MUTATION_RATE:
            j = random.randint(0, len(route) - 1)
            route[i], route[j] = route[j], route[i]
    return route

def evolve_population(pop, cities):
    ranked = rank_population(pop, cities)
    selected = select(ranked)
    children = [selected[i] if i < ELITE_SIZE else crossover(random.choice(selected), random.choice(selected)) for i in range(POPULATION_SIZE)]
    return [mutate(child) for child in children]

def genetic_algorithm(cities):
    population = initialize_population()
    best_route = None
    best_distance = float('inf')

    for _ in range(MAX_GENERATIONS):
        population = evolve_population(population, cities)
        current_best = rank_population(population, cities)[0]
        current_distance = route_distance(current_best, cities)
        if current_distance < best_distance:
            best_distance = current_distance
            best_route = current_best

    return best_route, best_distance

def main():
    global cities
    cities = generate_cities()
    best_route, best_distance = genetic_algorithm(cities)
    print("Best route:", best_route)
    print("Distance:", round(best_distance, 2))

if __name__ == "__main__":
    main()


Best route: [7, 1, 0, 6, 2, 3, 8, 9, 4, 5]
Distance: 370.44


In [2]:
!pip install python-chess


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 45.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147775 sha256=e15f3956c881c30c175365b188b286cb726f58d81e43b48faeb3cad86166cd95
  Stored in directory: /root/.cache/pip/wheels/fb/5d/5c/59a62d8a695285e59ec9c1f66add6f8a9ac4152499a2be0113
Successfully built chess
